## Soft Label 분포 기반 데이터 재분할

이 노트북은 `05_softlabel_dataset.csv` 파일을 읽어, soft label의 분포를 최대한 유지하면서 데이터를 Train, Validation, Test 세트로 다시 분할합니다.

**분할 전략:**
1. 각 이미지의 soft label 값들 중 최대값을 찾습니다.
2. 최대값이 유일하면 해당 감정을, 최대값이 여러 개(동점)이면 조합된 감정(예: 'anger-sad')을 기준으로 삼습니다.
3. 이 기준('stratify_key')의 분포를 train, val, test 세트에서 동일하게 유지하도록 계층적으로 샘플링합니다.

### 1. 라이브러리 및 데이터 로드

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 100)

# 데이터 파일 경로
csv_path = './05_softlabel_dataset.csv'

# 데이터 로드
df = pd.read_csv(csv_path)

print(f'데이터 형태: {df.shape}')
df.head()

데이터 형태: (8394, 16)


,phase,category,filename,img_path,exists,annot_A,annot_B,annot_C,soft_anger,soft_contempt,soft_disgust,soft_fear,soft_happy,soft_neutral,soft_sad,soft_surprise
0,train,anger,6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63...,./Data/EST_data/img_train/anger/6oj439e3fbcc52...,True,분노,분노,분노,0.500000,0.3,0.200000,0.000000,0.000000,0.0,0.000000,0.0
1,train,anger,65rsfe402042f34319e10128c1ab9614e2f967690a64a0...,./Data/EST_data/img_train/anger/65rsfe402042f3...,True,분노,분노,분노,0.500000,0.3,0.200000,0.000000,0.000000,0.0,0.000000,0.0
2,train,anger,b1cbe34734870cc11c33334e02bea93ac3a3b061caab62...,./Data/EST_data/img_train/anger/b1cbe34734870c...,True,상처,불안,슬픔,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.666667,0.0
3,train,anger,llfycc0aa29599cc63cace3610fdaaad3a99aab2ee38c9...,./Data/EST_data/img_train/anger/llfycc0aa29599...,True,분노,분노,분노,0.500000,0.3,0.200000,0.000000,0.000000,0.0,0.000000,0.0
4,train,anger,3hww73b70615461a7336d0383b53582f8bf804f6e0f30d...,./Data/EST_data/img_train/anger/3hww73b7061546...,True,기쁨,분노,분노,0.333333,0.2,0.133333,0.000000,0.333333,0.0,0.000000,0.0


### 2. 계층적 분할을 위한 키(Stratify Key) 생성 및 분포 확인

Soft label 컬럼들에서 최대값을 가지는 감정(들)을 찾아 새로운 키를 생성합니다.

In [22]:
# Soft label 컬럼 목록을 정의
soft_label_columns = [col for col in df.columns if col.startswith('soft_')]
print('Soft Label 컬럼:', soft_label_columns)

def get_stratify_key(row):
    '''한 행(row)을 입력받아 soft label의 최대값에 해당하는 키를 반환하는 함수'''
    # soft label 값들만 추출
    soft_labels = row[soft_label_columns]
    # 최대값 찾기
    max_value = soft_labels.max()
    
    # 최대값을 가진 컬럼(감정)들을 찾기
    max_labels = soft_labels[soft_labels == max_value].index.tolist()
    
    # 컬럼명에서 'soft_' 접두사 제거 및 알파벳순 정렬
    cleaned_labels = sorted([label.replace('soft_', '') for label in max_labels])
    
    # 하이픈(-)으로 연결하여 최종 키 생성
    return '-'.join(cleaned_labels)

# 모든 행에 함수를 적용하여 'stratify_key' 컬럼 생성
df['stratify_key'] = df.apply(get_stratify_key, axis=1)

key_distribution = df['stratify_key'].value_counts()
print('Stratify Key 원본 분포:')
print(key_distribution)

Soft Label 컬럼: ['soft_anger', 'soft_contempt', 'soft_disgust', 'soft_fear', 'soft_happy', 'soft_neutral', 'soft_sad', 'soft_surprise']
Stratify Key 원본 분포:
stratify_key
happy                 2148
sad                   1957
surprise              1414
anger                 1139
fear                   644
neutral                274
anger-fear             210
anger-sad              205
fear-sad               114
fear-neutral-sad        81
neutral-sad             59
anger-neutral           53
fear-neutral            27
anger-happy             21
happy-sad               11
fear-happy-sad          11
happy-neutral            9
fear-happy-neutral       6
happy-neutral-sad        6
fear-happy               5
Name: count, dtype: int64


### 3. 희귀 클래스 식별 및 Train / Validation / Test 분리

분할 후 temp_df에 1개만 남을 가능성이 있는 희귀 키를 식별합니다.
test_size=0.2를 곱했을 때 결과가 1이 될 가능성이 있는 클래스들입니다.
즉, 전체 샘플 수가 5~10개인 경우 (5~10 * 0.2 = 1~2) 입니다.

In [24]:
# 분할 후 temp_df에 1개만 남을 가능성이 있는 희귀 키를 식별
rare_keys_for_split_issue = key_distribution[key_distribution < 10].index.tolist()
print(f"분할 시 1개가 될 가능성이 있는 희귀 키: {rare_keys_for_split_issue}")

# 희귀 키와 나머지 데이터를 분리
df_rare = df[df['stratify_key'].isin(rare_keys_for_split_issue)]
df_main = df[~df['stratify_key'].isin(rare_keys_for_split_issue)]
print(f"메인 데이터셋 크기: {len(df_main)}, 희귀 데이터셋 크기: {len(df_rare)}")

분할 시 1개가 될 가능성이 있는 희귀 키: ['happy-neutral', 'fear-happy-neutral', 'happy-neutral-sad', 'fear-happy']
메인 데이터셋 크기: 8368, 희귀 데이터셋 크기: 26


In [36]:
# 메인 데이터셋을 먼저 분리
X = df_main.drop('stratify_key', axis=1)
y = df_main['stratify_key']

train_df, temp_df = train_test_split(
    df_main, 
    test_size=(0.2), 
    random_state=42, 
    stratify=y
)

# 희귀 데이터셋을 train_df에 합침
# 희귀 키를 train에 통합하여 안정적인 분할을 보장하고, 모델이 희귀 데이터로 학습할 기회를 제공
# 학습 기회를 제공하나, 전체 성능 지표를 왜곡할 수 있는 검증/테스트 과정에서는 제외, 모델의 핵심 성능을 더 객관적으로 측정
# 모든 데이터를 평가에 사용해야 한다는 원칙을 일부 양보하는 대신, 서비스의 주요 감정(happy, sad, anger 등)에 대한 모델의 성능을 최우선으로 확보
train_df = pd.concat([train_df, df_rare])
print(f"희귀 데이터셋을 train_df에 통합했습니다. 최종 train 세트 크기: {len(train_df)}")

# temp 세트를 val / test 세트로 분리
val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.5,
    random_state=42, 
    stratify=temp_df['stratify_key']
)

print(f'\n전체 데이터: {len(df)}개')
print(f'Train 세트: {len(train_df)}개')
print(f'Validation 세트: {len(val_df)}개')
print(f'Test 세트: {len(test_df)}개')

# 분할된 세트의 'stratify_key' 분포 확인
train_distribution = train_df['stratify_key'].value_counts()
train_distribution_norm = train_df['stratify_key'].value_counts(normalize=True)

val_distribution = val_df['stratify_key'].value_counts()
val_distribution_norm = val_df['stratify_key'].value_counts(normalize=True)

test_distribution = test_df['stratify_key'].value_counts()
test_distribution_norm = test_df['stratify_key'].value_counts(normalize=True)

# 결측값(NaN)은 0으로 채우고, 인덱스(감정)를 기준으로 정렬
distribution_df = pd.concat(
    [train_distribution, val_distribution, test_distribution, train_distribution_norm, val_distribution_norm, test_distribution_norm],
    axis=1,
    keys=['Train', 'Val', 'Test', 'Train (Norm)', 'Val (Norm)', 'Test (Norm)']
).fillna(0).astype(float)

# 표를 보기 좋게 출력
print("\n데이터 세트별 'stratify_key' 감정 분포:")
print(distribution_df.to_markdown())

희귀 데이터셋을 train_df에 통합했습니다. 최종 train 세트 크기: 6720

전체 데이터: 8394개
Train 세트: 6720개
Validation 세트: 837개
Test 세트: 837개

데이터 세트별 'stratify_key' 감정 분포:
| stratify_key       |   Train |   Val |   Test |   Train (Norm) |   Val (Norm) |   Test (Norm) |
|:-------------------|--------:|------:|-------:|---------------:|-------------:|--------------:|
| happy              |    1718 |   215 |    215 |    0.255655    |   0.25687    |    0.25687    |
| sad                |    1566 |   196 |    195 |    0.233036    |   0.23417    |    0.232975   |
| surprise           |    1131 |   141 |    142 |    0.168304    |   0.168459   |    0.169654   |
| anger              |     911 |   114 |    114 |    0.135565    |   0.136201   |    0.136201   |
| fear               |     515 |    64 |     65 |    0.0766369   |   0.0764636  |    0.0776583  |
| neutral            |     219 |    28 |     27 |    0.0325893   |   0.0334528  |    0.0322581  |
| anger-fear         |     168 |    21 |     21 |    0.025       |   0.0

### 6. 저장

In [40]:
# 원본 DataFrame에 새로운 phase 할당
df.loc[train_df.index, 'new_phase'] = 'train'
df.loc[val_df.index, 'new_phase'] = 'val'
df.loc[test_df.index, 'new_phase'] = 'test'

# 'new_phase'로 'phase' 덮어쓰기
df['phase'] = df['new_phase']

# 'new_phase' 컬럼 삭제
df.drop(columns=['new_phase'], inplace=True)

# 결과 저장
output_filename = '06_softlabel_dataset_resplit.csv'
df.to_csv(output_filename, index=False)

print(f'분할 결과가 {output_filename} 파일로 저장되었습니다.')
df[['filename', 'phase', 'stratify_key']].head()

분할 결과가 06_softlabel_dataset_resplit.csv 파일로 저장되었습니다.


,filename,phase,stratify_key
0,6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63...,train,anger
1,65rsfe402042f34319e10128c1ab9614e2f967690a64a0...,test,anger
2,b1cbe34734870cc11c33334e02bea93ac3a3b061caab62...,train,sad
3,llfycc0aa29599cc63cace3610fdaaad3a99aab2ee38c9...,train,anger
4,3hww73b70615461a7336d0383b53582f8bf804f6e0f30d...,train,anger-happy
